In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(1)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)

region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]/option[5]')
dong_1 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[9]').text
dong_2 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[14]').text
dong_3 = driver.find_element_by_xpath('//*[@id="DONG_NM"]/option[25]').text

dong_list = [dong_1, dong_2, dong_3]

for dong in dong_list:
    region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]/option[5]').click()
    dongs = driver.find_element_by_id('DONG_NM')
    dongs.send_keys(dong)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="glopopd_excel"]/span').click()
    time.sleep(2)

driver.quit()

In [3]:
import pandas as pd
from glob import glob

In [4]:
glob('./station_dong_data/지역*.xls')

['./station_dong_data\\지역_위치별(주유소) (1).xls',
 './station_dong_data\\지역_위치별(주유소) (2).xls',
 './station_dong_data\\지역_위치별(주유소).xls']

In [5]:
df = pd.read_excel('./station_dong_data/지역_위치별(주유소).xls', header=2)
df.head()

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000
1,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000


In [6]:
station_files = glob('./station_dong_data/지역*.xls')
xls_raw = []

for file_name in station_files:
    xls = pd.read_excel(file_name, header=2) #header=2라면 위의 2줄은 건너뛰고 3번째줄부터 데이터를 읽는다.
    xls_raw.append(xls)

dong_station = pd.concat(xls_raw, ignore_index=True)
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000


In [7]:
import requests
from urllib.parse import urlparse, quote
import folium

In [8]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [9]:
from tqdm import tqdm_notebook

lat_list=[]; lng_list=[]

for i in tqdm_notebook(dong_station.index):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(dong_station['주소'][i])
    result = requests.get(url, headers={"Authorization": "KakaoAK "+ kmap_key}).json()
    
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [10]:
dong_station['lat'] = lat_list
dong_station['lng'] = lng_list
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유,lat,lng
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-,37.572898,126.823440
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-,37.566243,126.840421
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-,37.540768,126.841472
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-,37.531562,126.856035
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-,37.530055,126.862834
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100,37.529645,126.859835
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-,37.527320,126.842879
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-,37.540242,126.841164
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000,37.557842,126.842395
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000,37.551803,126.836681


In [11]:
dong_station = dong_station.sort_values(by='휘발유')
dong_station

,지역,상호,주소,상표,전화번호,셀프여부,고급휘발유,휘발유,경유,실내등유,lat,lng
2,서울특별시,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),알뜰주유소,02-2691-5151,Y,-,1243,1043,-,37.540768,126.841472
3,서울특별시,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),알뜰주유소,02-2601-0077,N,-,1243,1043,-,37.531562,126.856035
4,서울특별시,목화주유소,서울 강서구 국회대로 251 (화곡동),SK에너지,02-2654-4564,Y,-,1266,1072,-,37.530055,126.862834
5,서울특별시,지에스칼텍스㈜ 경인고속주유소,서울 강서구 국회대로 225 (화곡동),GS칼텍스,02-2608-5151,Y,1416,1266,1072,1100,37.529645,126.859835
6,서울특별시,신화곡주유소,서울 강서구 국회대로 71,S-OIL,02-2611-3161,Y,-,1315,1115,-,37.527320,126.842879
7,서울특별시,화곡현대주유소,서울 강서구 강서로 151 (화곡동),SK에너지,02-2607-1942,Y,1455,1326,1126,-,37.540242,126.841164
8,서울특별시,대성석유(주)직영관문주유소,서울 강서구 공항대로 314,GS칼텍스,02-3665-2051,N,1599,1358,1187,1000,37.557842,126.842395
9,서울특별시,우장산주유소,서울 강서구 강서로 292 (내발산동),SK에너지,02-3664-7163,N,-,1358,1187,1000,37.551803,126.836681
0,서울특별시,대성석유(주)방화대성주유소,서울 강서구 양천로 176,GS칼텍스,02-3663-7781,Y,1599,1365,1175,-,37.572898,126.823440
1,서울특별시,대성석유(주) 마곡대성주유소,서울 강서구 강서로 457,GS칼텍스,02-3664-7781,Y,1599,1365,1185,-,37.566243,126.840421


In [1]:
map = folium.Map(location=[dong_station['lat'].mean(), dong_station['lng'].mean()], zoom_start=13)

for i in dong_station.index:
    if i == 0:
        folium.CircleMarker([dong_station['lat'][i], dong_station['lng'][i]],
                            radius=15,
                            color='#CD3181',
                            fill_color='#CD3181',
                            tooltip=(f"{dong_station['휘발유'][i]}"),
                            popup=f"{dong_station['상호'][i]}"
                            ).add_to(map)
    else:
        folium.CircleMarker([dong_station['lat'][i], dong_station['lng'][i]],
                            radius=15,
                            color='#4B89DC',
                            fill_color='#4B89DC',
                            tooltip=(f"{dong_station['휘발유'][i]}"),
                            popup=f"{dong_station['상호'][i]}"
                            ).add_to(map)
map

NameError: name 'folium' is not defined